[README](https://dumps.wikimedia.org/other/mediawiki_history/readme.html)
[schema doc](https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Edits/Mediawiki_history_dumps#Technical_Documentation)

In [1]:
# TODO: explore a single page

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

### Parse Raw Data

In [5]:
raw_cols = [
    'wiki_db', # only downloaded enwiki
    'event_entity',
    'event_type',
    'event_timestamp',
    'event_comment_escaped',
    'event_user_id', # id of editor, null if anon
    'event_user_text_historical_escaped', # username or ip of editor
    'event_user_text_escaped', # current username, null for anon
    'event_user_blocks_historical_string',
    'event_user_blocks_string',
    'event_user_groups_historical_string',
    'event_user_groups_string',
    'event_user_is_bot_by_historical_string', # array
    'event_user_is_bot_by_string', # array
    'event_user_is_created_by_self',
    'event_user_is_created_by_system',
    'event_user_is_created_by_peer', 
    'event_user_is_anonymous', # true if the revision was deleted
    'event_user_registration_timestamp',
    'event_user_creation_timestamp',
    'event_user_first_edit_timestamp',
    'event_user_revision_count',
    'event_user_seconds_since_previous_revision',
    'page_id',
    'page_title_historical_escaped',
    'page_title_escaped',
    'page_namespace_historical', 
    'page_namespace_is_content_historical',
    'page_namespace', 
    'page_namespace_is_content', 
    'page_is_redirect',
    'page_is_deleted',
    'page_creation_timestamp',
    'page_first_edit_timestamp', # can be before creation in restore/merge cases
    'page_revision_count',
    'page_seconds_since_previous_revision',
    'user_id',
    'user_text_historical_escaped', 
    'user_text_escaped',
    'user_blocks_historical_string', 
    'user_blocks_string',
    'user_groups_historical_string',
    'user_groups_string', 
    'user_is_bot_by_historical_string',
    'user_is_bot_by_string', 
    'user_is_created_by_self',
    'user_is_created_by_system',
    'user_is_created_by_peer', 
    'user_is_anonymous',
    'user_registration_timestamp',
    'user_creation_timestamp', 
    'user_first_edit_timestamp',
    'revision_id',
    'revision_parent_id',
    'revision_minor_edit',
    'revision_deleted_parts_string', # array
    'revision_deleted_parts_are_suppressed',
    'revision_text_bytes',
    'revision_text_bytes_diff', 
    'revision_text_sha1',
    'revision_content_model',
    'revision_content_format',
    'revision_is_deleted_by_page_deletion',
    'revision_deleted_by_page_deletion_timestamp',
    'revision_is_identity_reverted', # whether or not this is reverted by a future revision
    'revision_first_identity_reverting_revision_id', # id of the revision that reverts this one
    'revision_seconds_to_identity_revert',
    'revision_is_identity_revert',
    'revision_is_from_before_page_creation',
    'revision_tags_string', # array
]

str_cols = [
    'event_entity',
    'event_type',
    'event_comment_escaped',
    'event_user_id', # id of editor, null if anon
    'event_user_text_historical_escaped', # username or ip of editor
    'event_user_text_escaped', # current username, null for anon
    'event_user_is_bot_by_historical_string', # array
    'event_user_is_bot_by_string', # array
    'page_title_historical_escaped',
    'page_title_escaped',
    'revision_deleted_parts_string', # array
    'revision_tags_string', # array
]

int_cols = [
    'event_user_revision_count',
    'event_user_seconds_since_previous_revision',
    'page_id',
    'page_namespace_historical',
    'page_namespace',
    'page_revision_count',
    'page_seconds_since_previous_revision',
    'revision_id',
    'revision_parent_id',
    'revision_first_identity_reverting_revision_id',
    'revision_seconds_to_identity_revert',
    'revision_text_bytes',
    'revision_text_bytes_diff',

]

bool_cols = [
    'event_user_is_anonymous',
    'page_namespace_is_content_historical',
    'page_namespace_is_content',
    'page_is_redirect',
    'page_is_deleted',
    'revision_minor_edit',
    'revision_deleted_parts_are_suppressed',
    'revision_is_deleted_by_page_deletion',
    'revision_is_from_before_page_creation',
    'revision_is_identity_reverted',
]

date_cols = [
    'event_timestamp',
    'event_user_registration_timestamp',
    'event_user_creation_timestamp',
    'event_user_first_edit_timestamp',
    'page_creation_timestamp',
    'page_first_edit_timestamp',
    'revision_deleted_by_page_deletion_timestamp',
]

cols_to_keep = sorted(str_cols + int_cols + bool_cols + date_cols, key=lambda x: raw_cols.index(x))

df = pd.read_csv(
    '2022-12.enwiki.2022-12.tsv', 
    sep='\t',
    # nrows=10,
    # skiprows=3_979_125,
    usecols=[raw_cols.index(c) for c in cols_to_keep],
    dtype=str, 
    true_values=['true'],
    false_values=['false'],
    header=None,
)
df.columns = cols_to_keep
df = df[df['event_entity'] != 'user']
df = df[df['page_namespace_is_content'] == 'true']
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors='coerce')
for c in int_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df.to_parquet('parsed_december_page_edits.parquet')

In [6]:
df = pd.read_parquet('parsed_december_page_edits.parquet')

In [7]:
df.head()

,event_entity,event_type,event_timestamp,event_comment_escaped,event_user_id,event_user_text_historical_escaped,event_user_text_escaped,event_user_is_bot_by_historical_string,event_user_is_bot_by_string,event_user_is_anonymous,event_user_registration_timestamp,event_user_creation_timestamp,event_user_first_edit_timestamp,event_user_revision_count,event_user_seconds_since_previous_revision,page_id,page_title_historical_escaped,page_title_escaped,page_namespace_historical,page_namespace_is_content_historical,page_namespace,page_namespace_is_content,page_is_redirect,page_is_deleted,page_creation_timestamp,page_first_edit_timestamp,page_revision_count,page_seconds_since_previous_revision,revision_id,revision_parent_id,revision_minor_edit,revision_deleted_parts_string,revision_deleted_parts_are_suppressed,revision_text_bytes,revision_text_bytes_diff,revision_is_deleted_by_page_deletion,revision_deleted_by_page_deletion_timestamp,revision_is_identity_reverted,revision_first_identity_reverting_revision_id,revision_seconds_to_identity_revert,revision_is_from_before_page_creation,revision_tags_string
4,revision,create,2022-12-01 00:00:00,/* Men's game results */,25667,Dale Arnett,Dale Arnett,None,None,false,2003-10-12 04:17:07,NaT,2003-10-12 04:17:07,280693.0,100.0,37511171,Battle_of_Brooklyn_(college_rivalry),Battle_of_Brooklyn_(college_rivalry),0,true,0,true,false,false,2015-11-08 17:30:39,2012-11-01 07:39:24,387.0,100.0,1.124875e+09,1.124874e+09,true,None,false,34326.0,-10.0,false,NaT,false,NaN,NaN,false,wikieditor
7,revision,create,2022-12-01 00:00:00,None,9777540,Pelmeen10,Pelmeen10,None,None,false,2009-05-27 19:09:58,2009-05-27 19:10:00,2009-06-04 12:51:22,54964.0,24.0,7122751,Ginásio_Clube_Vilacondense,Ginásio_Clube_Vilacondense,0,true,0,true,false,false,2006-09-23 15:13:24,2006-09-23 15:13:24,18.0,24.0,1.124875e+09,1.124875e+09,true,None,false,1342.0,-1.0,false,NaT,false,NaN,NaN,false,None
13,revision,create,2022-12-01 00:00:00,Added link for mosque,None,86.18.158.129,86.18.158.129,None,None,true,NaT,NaT,NaT,NaN,NaN,483698,"Normanton,_Derby","Normanton,_Derby",0,true,0,true,false,false,2004-02-23 05:43:21,2004-02-23 05:43:21,238.0,129.0,1.124875e+09,1.124874e+09,false,None,false,13256.0,11.0,false,NaT,false,NaN,NaN,false,"mobile web edit,mobile edit,visualeditor"
15,revision,create,2022-12-01 00:00:01,None,24593355,Stvbastian,Stvbastian,None,None,false,2015-03-30 17:19:34,2015-03-30 17:19:35,2016-07-27 01:41:40,42137.0,44.0,51928380,Chen_Yufei,Chen_Yufei,0,true,0,true,false,false,2016-10-10 01:34:30,2016-10-10 01:34:30,424.0,126428.0,1.124875e+09,1.124583e+09,false,None,false,58551.0,0.0,false,NaT,false,NaN,NaN,false,wikieditor
17,revision,create,2022-12-01 00:00:01,New episode now airing.,20593931,BrickMaster02,BrickMaster02,None,None,false,2014-01-19 02:29:38,2014-01-19 02:29:40,2014-01-19 02:34:34,17767.0,3016.0,68737747,NFL_Slimetime,NFL_Slimetime,0,true,0,true,false,false,2021-09-16 12:53:19,2021-09-16 12:53:19,122.0,160099.0,1.124875e+09,1.124519e+09,true,None,false,22241.0,0.0,false,NaT,false,NaN,NaN,false,wikieditor


In [8]:
df['event_entity'].value_counts()

revision    3143510
page         124957
Name: event_entity, dtype: int64

In [9]:
df['event_type'].value_counts()

create         3198459
create-page      46993
move             13167
delete            9507
restore            306
merge               35
Name: event_type, dtype: int64

In [10]:
df['page_title_escaped'].value_counts().head(20)

Bigg_Boss_(Tamil_season_6)                        2387
Deaths_in_December_2022                           2322
List_of_most-liked_Instagram_posts                1986
Twitter_Files                                     1430
Avatar:_The_Way_of_Water                          1301
2023_PDC_World_Darts_Championship                 1301
Ronald_Reagan                                     1170
FIFA_World_Cup_records_and_statistics             1023
Bigg_Boss_(Telugu_season_6)                        994
2022_FIFA_World_Cup                                976
Joe_Martin_(orangutan)                             871
George_Santos                                      863
2022_FIFA_World_Cup_knockout_stage                 771
2022_FIFA_World_Cup_final                          754
Late_December_2022_North_American_winter_storm     753
Croatia_national_football_team                     751
Wye_College                                        733
Indiewood                                          628
List_of_fo

Start with a summary of event types 
- first and last occurrance
- sorted by frequency
- volume over final 30 days (configurable)
- 
- [checkbox to exclude event? there are almost always some garbage/noise ones]

Visuals to start with
- Clickable plotly-esq chart to see relative volume of events over time (this is going to need to consider a huge number of events, probably a good starting point!)
- Sankey charts with great configure-ability of what events are included, duplication, and start/end points

In [50]:
df.groupby('event_type').count()


,event_entity,event_timestamp,event_comment_escaped,event_user_id,event_user_text_historical_escaped,event_user_text_escaped,event_user_is_bot_by_historical_string,event_user_is_bot_by_string,event_user_is_anonymous,event_user_registration_timestamp,event_user_creation_timestamp,event_user_first_edit_timestamp,event_user_revision_count,event_user_seconds_since_previous_revision,page_id,page_title_historical_escaped,page_title_escaped,page_namespace_historical,page_namespace_is_content_historical,page_namespace,page_namespace_is_content,page_is_redirect,page_is_deleted,page_creation_timestamp,page_first_edit_timestamp,page_revision_count,page_seconds_since_previous_revision,revision_id,revision_parent_id,revision_minor_edit,revision_deleted_parts_string,revision_deleted_parts_are_suppressed,revision_text_bytes,revision_text_bytes_diff,revision_is_deleted_by_page_deletion,revision_deleted_by_page_deletion_timestamp,revision_is_identity_reverted,revision_first_identity_reverting_revision_id,revision_seconds_to_identity_revert,revision_is_from_before_page_creation,revision_tags_string
event_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
create,3198459,3198459,2459908,2572366,3197654,3197652,170392,170389,3197654,2564965,2349640,2572364,2518297,2491412,3198459,3198459,3198459,3198459,3198459,3198459,3198459,3176016,3198459,3197782,3195821,3143510,3087479,3143510,3143510,3143510,4992,3143510,3143466,3143455,3143510,18385,3143510,287173,287173,3143510,2796510
create-page,46993,46993,0,46929,46992,46992,1381,1381,46992,46848,42025,46929,0,0,46993,46993,46993,46993,46993,46993,46993,44375,46993,46908,46889,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
delete,9507,9507,0,9507,9507,9507,13,13,9507,9499,8304,9507,0,0,9507,9507,9507,9507,9507,9507,9507,123,9507,8781,7274,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
merge,35,35,0,35,35,35,0,0,35,35,33,35,0,0,35,35,35,35,35,35,35,32,35,35,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
move,13167,13167,0,13167,13167,13167,358,358,13167,13151,11131,13167,0,0,13167,13167,13167,13167,13167,13167,13167,12744,13167,13157,13140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
restore,306,306,0,306,306,306,0,0,306,305,113,306,0,0,306,306,306,306,306,306,306,285,306,304,302,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
df['event_name'] = df['event_entity'] + '_' + df['event_type']
subset = df.loc[:, [
    'event_name', 
    'event_timestamp',
    'event_user_text_historical_escaped', 
    'event_user_is_bot_by_historical_string',
    'event_user_revision_count',
    'event_user_seconds_since_previous_revision',
    'page_id',
    'page_title_historical_escaped',
    'page_revision_count',
    'revision_text_bytes',
    'revision_is_identity_reverted',
    'revision_seconds_to_identity_revert', # need to re-build to get this one
    'revision_tags_string'
]]

In [ ]:
subset['indicator'] = 1

In [154]:
# TODO: make event name and event timestamp magic words

def summarize(df):

    metadata = dict() 

    for event_name, row_counts in df.groupby('event_name').count().iterrows():
        metadata[event_name] = dict()
        event_count = int(row_counts.loc['indicator'])
        metadata[event_name]['count'] = event_count
        metadata[event_name]['attributes'] = {
            k: {
                'count': v,
                'null_pct': int(100 - round(v/event_count*100)),
                
            } for k,v in row_counts.items() if v > 0 and k != 'indicator'
        }
    
    for event_name, event_metadata in metadata.items():
        for col_name in event_metadata['attributes'].keys():
            col_type = df[col_name].dtype
            subset = df[df['event_name'] == event_name][col_name]
            event_metadata['attributes'][col_name]['type'] = str(col_type)
            if str(col_type) == 'object':
                event_metadata['attributes'][col_name]['top_values'] = subset.value_counts().head(5).to_dict()
            elif str(col_type) == 'datetime64[ns]' or pd.api.types.is_numeric_dtype(col_type):
                event_metadata['attributes'][col_name]['range'] = [subset.min(), subset.max()]
    
    return metadata



In [151]:
metadata = summarize(subset)

In [152]:
def default_serializer(x):
    if type(x) == np.int64:
        return int(x)
    elif type(x) == np.float64:
        return float(x)
    elif type(x) == pd.Timestamp:
        return x.isoformat()

In [153]:
with open('metadata.json', 'w') as f:
    json.dump(metadata, f, default=default_serializer)